In [ ]:
# Kaggle Boilerplate & Environment Check
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# Load and Preprocess Photos (Resize to 256x256)
from glob import glob
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def load_images(image_paths, size=(256, 256)):
    images = []
    for path in image_paths:
        img = load_img(path, target_size=size)
        img = img_to_array(img)
        img = (img / 127.5) - 1
        images.append(img)
    return np.array(images)

photo_paths = glob('/kaggle/input/gan-getting-started/photo_jpg/*.jpg')[:8000]  # Limit to 8000
photos = load_images(photo_paths)
print("Loaded", len(photos), "photos")


In [ ]:
# Define and Load Pretrained Generator Model
import tensorflow as tf
from tensorflow.keras import layers, Model

def build_generator():
    inputs = layers.Input(shape=(256, 256, 3))
    x = layers.Conv2D(64, (4, 4), strides=2, padding='same')(inputs)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2D(128, (4, 4), strides=2, padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2DTranspose(64, (4, 4), strides=2, padding='same')(x)
    x = layers.ReLU()(x)
    x = layers.Conv2DTranspose(3, (4, 4), strides=2, padding='same', activation='tanh')(x)
    return Model(inputs, x, name="Generator")

gen = build_generator()
# gen.load_weights('/kaggle/input/your-model-folder/gen_weights.h5')
print("✅ Generator model is ready.")


In [ ]:
#  Generate Monet-Style Images
from PIL import Image

output_dir = "/kaggle/working/generated_images"
os.makedirs(output_dir, exist_ok=True)

for i in range(7000):  # Must be between 7000–10000
    idx = i % len(photos)
    sample = np.expand_dims(photos[idx], axis=0)
    fake = gen(sample, training=False).numpy()[0]
    fake = ((fake + 1) * 127.5).astype(np.uint8)
    img = Image.fromarray(fake)
    img = img.resize((256, 256))
    img.save(f"{output_dir}/{i:05}.jpg")

print("✅ 7000 Monet-style images generated.")


In [ ]:
# Zip for Submission
import zipfile

zipf = zipfile.ZipFile("/kaggle/working/images.zip", 'w', zipfile.ZIP_DEFLATED)
for root, _, files in os.walk(output_dir):
    for file in files:
        zipf.write(os.path.join(root, file), file)
zipf.close()

print("✅ images.zip ready for submission.")
